In [1]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("OpenAssistant/oasst1")

Found cached dataset parquet (C:/Users/mathew.kenny/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-2960c57d7e52ab15/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
train_dataset = dataset['train']      # len(train)=84437 (95%)
val_dataset = dataset['validation']   # len(val)=4401 (5%)

In [3]:
# Creating a DataFrame with the keys 'text', 'parent_id', 'message_id' and 'role'
df = pd.DataFrame({'message_tree_id': val_dataset['message_tree_id'], 'parent_id': val_dataset['parent_id'], 'message_id': val_dataset['message_id'], 'role': val_dataset['role'], 'text': val_dataset['text']})
df

,message_tree_id,parent_id,message_id,role,text
0,68489e5c-978f-4ad7-a849-39a741fb5ae7,None,68489e5c-978f-4ad7-a849-39a741fb5ae7,prompter,"Напиши функцию на языке swift, которая сортиру..."
1,68489e5c-978f-4ad7-a849-39a741fb5ae7,68489e5c-978f-4ad7-a849-39a741fb5ae7,a38b3d1c-256f-47c5-ab68-c9ab766bad84,assistant,"Вот функция, которая сортирует массив целых чи..."
2,68489e5c-978f-4ad7-a849-39a741fb5ae7,68489e5c-978f-4ad7-a849-39a741fb5ae7,9cf52b01-0582-46aa-9ad2-70827dce87ed,assistant,"Вот функция на языке Swift, которая сортирует ..."
3,68489e5c-978f-4ad7-a849-39a741fb5ae7,68489e5c-978f-4ad7-a849-39a741fb5ae7,9685d79e-16d7-4f1b-b68f-a1ff73e87bef,assistant,func sortAndPrintArray(array: [Int]) {\n le...
4,afee0474-c84a-4ae2-a9c9-8d575256312e,None,afee0474-c84a-4ae2-a9c9-8d575256312e,prompter,"Inventa un monstruo altamente complejo, descrí..."
...,...,...,...,...,...
4396,d6545af3-e409-4c0f-aa62-5e0ad9748424,74cef2d0-2a22-4409-acdc-799e0ce32f3d,7da63f52-1496-4449-9402-05a3cf8217c8,prompter,今日はどんな気分？
4397,d6545af3-e409-4c0f-aa62-5e0ad9748424,d6545af3-e409-4c0f-aa62-5e0ad9748424,a9f68e53-4335-4d27-89c5-4d0a2bc857b3,assistant,こんにちは！
4398,d6545af3-e409-4c0f-aa62-5e0ad9748424,a9f68e53-4335-4d27-89c5-4d0a2bc857b3,88ad2577-484c-4e87-b4d2-e12665881012,prompter,自己紹介をお願いします。
4399,d6545af3-e409-4c0f-aa62-5e0ad9748424,88ad2577-484c-4e87-b4d2-e12665881012,ea15b999-1d90-4c34-bc3a-caa2522a7182,assistant,私は Open Assistant です。名前はまだありません。


In [4]:
# Creating a new DF to store the prompts and responses
output_df = pd.DataFrame(columns=['prompt', 'response'])

# Collecting list of unqiue tree ids
tree_ids = set(df['message_tree_id'].unique())

# Looping to collect each prompt and response
for tree_id in tree_ids:
    temp_df = df[df['message_tree_id'] == tree_id]
    prompt_df = temp_df[temp_df["role"] == 'prompter']
    assistant_df = temp_df[temp_df["role"] == 'assistant']

    prompt_message_ids = prompt_df['message_id'].tolist()

    prompts = []
    responses = []

    for id in prompt_message_ids:
        prompt = prompt_df[prompt_df['message_id'] == id]['text'].iloc[0]
        # collecting responses for same question together
        response = assistant_df[assistant_df['parent_id'] == id]['text'].tolist()
        response = ' '.join(response)

        prompts.append(prompt)
        responses.append(response)
        
    # Adding context of first question to all other questions
    prompts = [prompts[0]] + [prompts[0] + element for element in prompts[1:]]
    
    # Storing to temp DF and then concatenating to main output DF
    temp_df = pd.DataFrame({'prompt': prompts, 'response': responses})
    output_df = pd.concat([output_df, temp_df], ignore_index=True)

In [5]:
output_df

,prompt,response
0,What are some options of foods that I can make...,1. Pizza (flat dough with tomato and cheese an...
1,What are some options of foods that I can make...,If you are looking for lactose-free options fo...
2,What are some options of foods that I can make...,
3,What are some options of foods that I can make...,
4,What are some options of foods that I can make...,"Before we get the ingredients together, you ha..."
...,...,...
1640,¿Por qué aprender Excel importante como ingeni...,"Sí, existen varias herramientas como Microsoft..."
1641,¿Por qué aprender Excel importante como ingeni...,
1642,¿Por qué aprender Excel importante como ingeni...,
1643,¿Por qué aprender Excel importante como ingeni...,Aprender Excel es importante para ingenieros i...


In [6]:
final_df = output_df[output_df['response'] != '']
final_df

,prompt,response
0,What are some options of foods that I can make...,1. Pizza (flat dough with tomato and cheese an...
1,What are some options of foods that I can make...,If you are looking for lactose-free options fo...
4,What are some options of foods that I can make...,"Before we get the ingredients together, you ha..."
5,How to code the snake game in C that will be s...,Here is a basic outline of how to code the sna...
6,How to code the snake game in C that will be s...,"Based on our previous conversation, it is uncl..."
...,...,...
1638,Quien invento la nitroglicerina?,La nitroglicerina fue inventada por el químico...
1639,¿Por qué aprender Excel importante como ingeni...,Aprender Microsoft Excel es importante para lo...
1640,¿Por qué aprender Excel importante como ingeni...,"Sí, existen varias herramientas como Microsoft..."
1643,¿Por qué aprender Excel importante como ingeni...,Aprender Excel es importante para ingenieros i...


In [7]:
# Write DataFrame to a JSON Lines file
with open('finetuning_data_val.jsonl', 'w') as file:
    for _, row in final_df.iterrows():
        file.write(row.to_json() + '\n')